In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
data_large=pd.read_csv("I:/Masters/SPRING 18/SPATIAL AND TEMPORAL/spatial/project/final/chicago_final_clean.csv",index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data_large.shape

(1709856, 110)

### For most algorithms, it will be hard to start from the whole dataframe (1.7 million records).
#### As a result, we will sample 5% of the record out and test different algorithms.

In [4]:
data=data_large.sample(80000)
data.shape

(80000, 110)

#### We will first drop columns we are not using. Though there are many features in this dataset, some information may not be useful at least for now, such as ID, FBI code and Block. Some features are overlapping with each other (such as ward, community area and district), and we will only keep one for simplicity (which also prevents us from including 100% correlated features, which can be bad for regression-based methods)

In [5]:
data=data.drop(["ID","Date","Block","IUCR","Description","Ward","Community.Area","FBI.Code"], axis=1)
data=data.drop(["X.Coordinate","Y.Coordinate","Year","Location","District","Primary.Type","time_24hour","Date_no_time"], axis=1)

In [6]:
data.head()

,Latitude,Longitude,severe,Primary.Type_ARSON,Primary.Type_ASSAULT,Primary.Type_BATTERY,Primary.Type_BURGLARY,Primary.Type_CRIM.SEXUAL.ASSAULT,Primary.Type_CRIMINAL.DAMAGE,Primary.Type_CRIMINAL.TRESPASS,Primary.Type_DECEPTIVE.PRACTICE,Primary.Type_GAMBLING,Primary.Type_HOMICIDE,Primary.Type_INTERFERENCE.WITH.PUBLIC.OFFICER,Primary.Type_INTIMIDATION,Primary.Type_KIDNAPPING,Primary.Type_LIQUOR.LAW.VIOLATION,Primary.Type_MOTOR.VEHICLE.THEFT,Primary.Type_NARCOTICS,Primary.Type_OFFENSE.INVOLVING.CHILDREN,Primary.Type_PROSTITUTION,Primary.Type_PUBLIC.PEACE.VIOLATION,Primary.Type_ROBBERY,Primary.Type_SEX.OFFENSE,Primary.Type_STALKING,Primary.Type_THEFT,Primary.Type_WEAPONS.VIOLATION,District_D1.0,District_D10.0,District_D11.0,District_D12.0,District_D14.0,District_D15.0,District_D16.0,District_D17.0,District_D18.0,District_D19.0,District_D2.0,District_D20.0,District_D22.0,District_D24.0,District_D25.0,District_D3.0,District_D4.0,District_D5.0,District_D6.0,District_D7.0,District_D8.0,District_D9.0,Timeblock_0,Timeblock_12,Timeblock_15,Timeblock_18,Timeblock_21,Timeblock_3,Timeblock_6,Timeblock_9,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday,closest_station,Income,HARDSHIP INDEX,Under18_over64,Unemployed,House_below_poverty,Location.Description_ALLEY,Location.Description_APARTMENT,Location.Description_BAR OR TAVERN,Location.Description_COMMERCIAL / BUSINESS OFFICE,Location.Description_CONVENIENCE STORE,Location.Description_CTA PLATFORM,Location.Description_CTA TRAIN,Location.Description_DEPARTMENT STORE,Location.Description_GAS STATION,Location.Description_GROCERY FOOD STORE,Location.Description_PARK PROPERTY,Location.Description_PARKING LOT/GARAGE(NON.RESID.),Location.Description_RESIDENCE,Location.Description_RESIDENCE PORCH/HALLWAY,Location.Description_RESIDENCE-GARAGE,Location.Description_RESIDENTIAL YARD (FRONT/BACK),Location.Description_RESTAURANT,"Location.Description_SCHOOL, PUBLIC, BUILDING","Location.Description_SCHOOL, PUBLIC, GROUNDS",Location.Description_SIDEWALK,Location.Description_SMALL RETAIL STORE,Location.Description_STREET,Location.Description_VACANT LOT/LAND,Location.Description_VEHICLE NON-COMMERCIAL
270162,41.695982,-87.596653,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0.463179,20588,51.0,38.6,22.8,21.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1343633,41.894179,-87.726050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1.771886,13781,85.0,38.0,17.3,33.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
471510,41.917774,-87.754621,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0.565694,15461,70.0,37.3,14.6,18.7,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
361181,41.861850,-87.736051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1.466856,12034,87.0,42.7,21.2,43.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1772445,41.758388,-87.616201,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0.773433,17285,66.0,41.0,23.0,29.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


#### Note that we are now predicting severe or not, so we will throw away the exact crime type for now. We will also break the "Location Description" into dummy (indicator) variable, a step that we forgot in our data cleanup part... We will also need to drop the "other" description for location because it's ambiguous. 

In [7]:
a=list(data.columns.values)
droplist=[]
for i in a:
    if i.startswith("Primary Type"):
        droplist.append(i)
data=data.drop(droplist, axis=1)

####  There are six features required normalization: Latitude, Longitude, closest_station, TMAX, TMIN, precipitation. Other features for our dataset are all categorical data and have all been converted to dummy variables (District, Time_block, Weekday, Location description, specific weather type). We will split the data into training set and test set, and normalize by themselves (to avoid contamination) 

In [8]:
from sklearn.cross_validation import train_test_split
itrain, itest = train_test_split(range(data.shape[0]), train_size=0.7)
mask=np.ones(data.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask=(mask==1)

C:\Users\weerd\AppData\Roaming\Python\Python36\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
print(data.shape)
data.head()

(80000, 94)


,Latitude,Longitude,severe,Primary.Type_ARSON,Primary.Type_ASSAULT,Primary.Type_BATTERY,Primary.Type_BURGLARY,Primary.Type_CRIM.SEXUAL.ASSAULT,Primary.Type_CRIMINAL.DAMAGE,Primary.Type_CRIMINAL.TRESPASS,Primary.Type_DECEPTIVE.PRACTICE,Primary.Type_GAMBLING,Primary.Type_HOMICIDE,Primary.Type_INTERFERENCE.WITH.PUBLIC.OFFICER,Primary.Type_INTIMIDATION,Primary.Type_KIDNAPPING,Primary.Type_LIQUOR.LAW.VIOLATION,Primary.Type_MOTOR.VEHICLE.THEFT,Primary.Type_NARCOTICS,Primary.Type_OFFENSE.INVOLVING.CHILDREN,Primary.Type_PROSTITUTION,Primary.Type_PUBLIC.PEACE.VIOLATION,Primary.Type_ROBBERY,Primary.Type_SEX.OFFENSE,Primary.Type_STALKING,Primary.Type_THEFT,Primary.Type_WEAPONS.VIOLATION,District_D1.0,District_D10.0,District_D11.0,District_D12.0,District_D14.0,District_D15.0,District_D16.0,District_D17.0,District_D18.0,District_D19.0,District_D2.0,District_D20.0,District_D22.0,District_D24.0,District_D25.0,District_D3.0,District_D4.0,District_D5.0,District_D6.0,District_D7.0,District_D8.0,District_D9.0,Timeblock_0,Timeblock_12,Timeblock_15,Timeblock_18,Timeblock_21,Timeblock_3,Timeblock_6,Timeblock_9,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday,closest_station,Income,HARDSHIP INDEX,Under18_over64,Unemployed,House_below_poverty,Location.Description_ALLEY,Location.Description_APARTMENT,Location.Description_BAR OR TAVERN,Location.Description_COMMERCIAL / BUSINESS OFFICE,Location.Description_CONVENIENCE STORE,Location.Description_CTA PLATFORM,Location.Description_CTA TRAIN,Location.Description_DEPARTMENT STORE,Location.Description_GAS STATION,Location.Description_GROCERY FOOD STORE,Location.Description_PARK PROPERTY,Location.Description_PARKING LOT/GARAGE(NON.RESID.),Location.Description_RESIDENCE,Location.Description_RESIDENCE PORCH/HALLWAY,Location.Description_RESIDENCE-GARAGE,Location.Description_RESIDENTIAL YARD (FRONT/BACK),Location.Description_RESTAURANT,"Location.Description_SCHOOL, PUBLIC, BUILDING","Location.Description_SCHOOL, PUBLIC, GROUNDS",Location.Description_SIDEWALK,Location.Description_SMALL RETAIL STORE,Location.Description_STREET,Location.Description_VACANT LOT/LAND,Location.Description_VEHICLE NON-COMMERCIAL
270162,41.695982,-87.596653,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0.463179,20588,51.0,38.6,22.8,21.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1343633,41.894179,-87.726050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1.771886,13781,85.0,38.0,17.3,33.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
471510,41.917774,-87.754621,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0.565694,15461,70.0,37.3,14.6,18.7,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
361181,41.861850,-87.736051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1.466856,12034,87.0,42.7,21.2,43.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1772445,41.758388,-87.616201,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0.773433,17285,66.0,41.0,23.0,29.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [10]:
#We have a list of continuous features, or in other words standardizable variables
STANDARDIZABLE=["Latitude", 'Longitude',"closest_station","House_below_poverty","Unemployed","Under18_over64","Income","Hardship"]

#Also create a list for indicator variable. We can do this by excluding the above continuous features from total features. 
INDICATOR=list(data.columns)
#We need to remove the response variable from our total list of features
INDICATOR.remove(u'severe')
#This is to create a list of indicator variables by excluding the continous features
for i in STANDARDIZABLE:
    INDICATOR.remove(i)
print(len(STANDARDIZABLE), len(INDICATOR))

ValueError: list.remove(x): x not in list

In [11]:
np.nan_to_num(data)
data = data.fillna(data.mean())


In [12]:
data.isnull().values.any()
data.isnull().sum().sum()

0

In [13]:
np.all(np.isfinite(data))
data = data.reset_index()

In [14]:

from sklearn.preprocessing import StandardScaler
#Standardize training set
data.loc[mask,STANDARDIZABLE]=StandardScaler().fit_transform(data.loc[mask,STANDARDIZABLE])
#Standardize test set
data.loc[~mask,STANDARDIZABLE]=StandardScaler().fit_transform(data.loc[~mask,STANDARDIZABLE])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').